Section 1: Import and Setup

In [1]:
import sys
import os
import numpy as np
from pathlib import Path
from datetime import datetime
import importlib  # For reloading modules during testing

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# First import the modules
import aicons.definitions.simple_bad_aicon
import aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor

# Now reload them for development/testing
importlib.reload(aicons.definitions.simple_bad_aicon)
importlib.reload(aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor)

# Import the classes after reloading
from aicons.definitions.simple_bad_aicon import SimpleBadAIcon
from aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor import MetaAdsSalesSensor

# Create the AIcon
aicon = SimpleBadAIcon("budget_allocation_aicon")
print(f"AIcon created: {aicon.name}")

AIcon created: budget_allocation_aicon


In [2]:
# Your Meta API credentials
access_token = "EAAZAn8wmq1IEBOZCz8oyDZBBgiazAgnQKIoAr4mFTbkV7jxi6t3APzOSxFybXNIkBgwQACdagbs5lFE8tpnNOBOOpWtS3KjZAdf9MNAlySpwEaDrX32oQwUTNmOZAaSXjT5Os5Q8YqRo57tXOUukB7QtcO8nQ8JuqrnnshCr7A0giynZBnJKfuPakrZBWoZD"  
ad_account_id = "act_252267674525035"     
campaign_id = "120218631288730217"           

# Create and add the Meta Ads Sensor
sensor = MetaAdsSalesSensor(
    name="meta_ads",
    reliability=0.9,
    access_token=access_token,
    ad_account_id=ad_account_id,
    campaign_id=campaign_id,
    api_version="v18.0",
    time_granularity="hour"
)

aicon.add_sensor("meta_ads", sensor)
print("Meta Ads sensor added")

# Get active ads to understand what we're working with
active_ads = sensor.get_active_ads()
print(f"Found {len(active_ads)} active ads")
for ad in active_ads:
    print(f"- Ad ID: {ad['ad_id']}, Name: {ad['ad_name']}")

# Check what factors were created
state_factors = aicon.brain.get_state_factors()
print(f"Number of factors: {len(state_factors)}")
print("Factor names:", ", ".join(state_factors.keys()))

✅ Successfully initialized Meta Ads API vv18.0 for account act_252267674525035
Auto-creating missing factor: purchases (continuous)
Creating continuous factor with TensorFlow: purchases
Added continuous factor with TensorFlow distribution: purchases
Auto-creating missing factor: add_to_carts (continuous)
Creating continuous factor with TensorFlow: add_to_carts
Added continuous factor with TensorFlow distribution: add_to_carts
Auto-creating missing factor: initiated_checkouts (continuous)
Creating continuous factor with TensorFlow: initiated_checkouts
Added continuous factor with TensorFlow distribution: initiated_checkouts
Auto-creating missing factor: ad_*_purchases (continuous)
Creating continuous factor with TensorFlow: ad_*_purchases
Added continuous factor with TensorFlow distribution: ad_*_purchases
Auto-creating missing factor: ad_*_add_to_carts (continuous)
Creating continuous factor with TensorFlow: ad_*_add_to_carts
Added continuous factor with TensorFlow distribution: ad_*_a

Create Action Space for Budget Allocation

In [3]:
# Define the total budget to allocate
total_budget = 1000.0  # $1,000 total budget

# Extract the ad IDs for the action space
ad_ids = [ad['ad_id'] for ad in active_ads]

# Create an action space for budget allocation
action_space = aicon.create_action_space(
    'budget_allocation',
    total_budget=total_budget,
    items=ad_ids,
    min_allocation_percent=0.0,
    max_allocation_percent=1.0,
    step_size=0.05,
    name="budget_allocation"
)

print(f"Created action space with {len(ad_ids)} ads and total budget ${total_budget}")

# Get information about the action space dimensions
dimensions = aicon.get_action_dimensions()
print(f"Action space dimensions: {dimensions['num_dimensions']}")
print(f"Dimension names: {dimensions['dimension_names']}")

# Generate example allocations
print("\nExample random budget allocations:")
for i in range(3):
    random_allocation = aicon.sample_action()
    print(f"Random allocation {i+1}:")
    
    # Get the values as a list
    values = [random_allocation[dim_name] for dim_name in dimensions['dimension_names']]
    
    # Print allocation for each ad
    for j, (ad_id, amount) in enumerate(zip(ad_ids, values)):
        print(f"  - Ad {ad_id}: ${amount:.2f}")

Created budget_allocation action space 'budget_allocation' with 5 dimensions
Created action space with 5 ads and total budget $1000.0
Action space dimensions: 5
Dimension names: ['ad_1_budget', 'ad_2_budget', 'ad_3_budget', 'ad_4_budget', 'ad_5_budget']

Example random budget allocations:
Random allocation 1:
  - Ad 120218902995700217: $250.00
  - Ad 120218902777970217: $150.00
  - Ad 120218902726170217: $100.00
  - Ad 120218902629840217: $300.00
  - Ad 120218905466570217: $200.00
Random allocation 2:
  - Ad 120218902995700217: $350.00
  - Ad 120218902777970217: $150.00
  - Ad 120218902726170217: $100.00
  - Ad 120218902629840217: $150.00
  - Ad 120218905466570217: $250.00
Random allocation 3:
  - Ad 120218902995700217: $150.00
  - Ad 120218902777970217: $100.00
  - Ad 120218902726170217: $150.00
  - Ad 120218902629840217: $250.00
  - Ad 120218905466570217: $350.00
